# Day 10: Syntax Scoring

You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:

Syntax error in navigation subsystem on line: all of them
All of them?! The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).

The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

If a chunk opens with (, it must close with ).
If a chunk opens with [, it must close with ].
If a chunk opens with {, it must close with }.
If a chunk opens with <, it must close with >.
So, () is a legal chunk that contains no other chunks, as is []. More complex but valid chunks include ([]), {()()()}, <([{}])>, [<>({}){}[([])<>]], and even (((((((((()))))))))).

## Setup

In [ ]:
var pairs = new Dictionary<char,char>{ ['<'] = '>', ['{'] = '}', ['['] = ']', ['(']= ')' };
var opens = pairs.Keys.ToArray();
var closes = pairs.Values.ToArray();

var score = new Dictionary<char, int> { [')'] = 3, [']'] = 57, ['}'] = 1197, ['>'] = 25137, ['-'] = 0 };

## Input

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-10.sample");
var input = File.ReadAllLines(@"day-10.input");

input.Length

106

## Part 1

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include (], {()()()>, (((()))}, and <([]){()}[{}]). Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

Stop at the first incorrect closing character on each corrupted line.

Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the first illegal character on the line and look it up in the following table:

```
  ): 3 points.
  ]: 57 points.
  }: 1197 points.
  >: 25137 points.
```

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

In [ ]:
char Parse(char[] input, Stack<char> seen) {
    if (input.Length == 0) return '-';
    var next = input[0];
    if (closes.Contains(next)) {
        var first = seen.Pop();
        if (next != pairs[first]) return next;
        return Parse(input[1..], seen);
    }
    if (opens.Contains(next)) {
        seen.Push(next);
        return Parse(input[1..], seen);
    }
    return '!';
}

input.Select(l => (l, Parse(l.ToCharArray(), new Stack<char>())))
    .Select(r => score[r.Item2])
    .Sum()

26397

## Part 2

Now, discard the corrupted lines. The remaining lines are incomplete.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out the sequence of closing characters that complete all open chunks in the line.

You can only use closing characters (), ], }, or >), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

Did you know that autocomplete tools also have contests? It's true! The score is determined by considering the completion string character-by-character. Start with a total score of 0. Then, for each character, multiply the total score by 5 and then increase the total score by the point value given for the character in the following table:

```
): 1 point.
]: 2 points.
}: 3 points.
>: 4 points.
```

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is 288957 because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

In [ ]:
static long Score(long prev, char close)
    => close switch {
            '(' => 1,
            '[' => 2,
            '{' => 3,
            '<' => 4,
            _ => 0
        } + prev * 5;

var incompletes = input
    .Select(l => (l.ToCharArray(), new Stack<char>()))
    .Where(t => Parse(t.Item1, t.Item2) == '-')
    .Select(t => t.Item2.Aggregate(0L, Score))
    .OrderBy(l => l)
    .ToArray();

incompletes[incompletes.Length / 2]

3122628974